In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os, sys
sys.path.append('/content/drive/MyDrive/CPD_BT')

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import random

import itertools

from bt_cpd import *

import time
import bisect

import pandas as pd

import statsmodels.api as sm
from sklearn import linear_model

from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)

In [4]:
T = 3
Delta = 1000
m = np.array([Delta] * T)
cp_truth = np.cumsum(m)[:T-1]

n = 100
beta = np.zeros((T, n))

t = 0.9
kappa = np.log(t / (1 - t))
delta = 1
beta_ref = get_beta_with_gap(n, delta)
beta_ref *= kappa / (np.max(beta_ref) - np.min(beta_ref))
beta[0] = beta_ref[:]
beta[1] = change_type(beta_ref, 1)
beta[2] = change_type(beta_ref, 2)

print(max(beta[0]) - min(beta[0]))

diff = np.zeros(T - 1)
for t in range(1, T):
    diff[t - 1] = np.sum(np.abs(beta[t] - beta[t - 1])**2)**0.5
print(diff)

2.1972245773362196
[12.81317951 11.09709382]


In [5]:
cp_truth

array([1000, 2000])

In [6]:
path = '/content/drive/MyDrive/CPD_BT/'
with open(path + 'data_n' + str(n) + '_Delta' + str(Delta) + '_K' + str(T - 1) + '.npy', 'rb') as f:
    X_train_list = np.load(f)
    Y_train_list = np.load(f)
    X_test_list = np.load(f)
    Y_test_list = np.load(f)

In [7]:
X_train_list.shape

(100, 3000, 100)

In [14]:
np.random.seed(0)

m_intervals = 50
grid_n = 200
gamma_list = [5, 20, 40]
lam_list = [0.1]

nt = Delta * T
B = 100

run_time_wbs = np.zeros(B)
loc_error_wbs = np.zeros(B)
K_wbs = np.zeros(B)

cp_best_list = []
param_best_list = []

for b in range(B):
    X_train = X_train_list[b]
    Y_train = Y_train_list[b]
    X_test = X_test_list[b]
    Y_test = Y_test_list[b]

    start_time = time.time()

    # calculate borda count
    Yx_train = np.zeros_like(X_train)
    Yx_test = np.zeros_like(X_test)
    for i in range(nt):
        Yx_train[i] = X_train[i] * (2 * Y_train[i] - 1)
        Yx_test[i] = X_test[i] * (2 * Y_test[i] - 1)

    wbs_fit = wbs_cv_mean(m_intervals, gamma_list, grid_n, smooth = 5, buffer = 5)
    cp_best, cp_val, cusum_val, threshold_best, grid = wbs_fit.fit(Yx_train, Yx_test)

    run_time_wbs[b] = time.time() - start_time
    loc_error_wbs[b] = cp_distance(cp_best, cp_truth)
    K_wbs[b] = len(cp_best)

    cp_best_list.append(cp_best)
    param_best_list.append(threshold_best)

    print(b)

print('---------- wbs -----------')
print("avg loc error: {0}, avg time: {1}".format(loc_error_wbs.mean(), run_time_wbs.mean()))
print("std loc error: {0}, std time: {1}".format(loc_error_wbs.std(), run_time_wbs.std()))
print('K < K*: {0}, K = K*: {1}, K > K*: {2}'.format(sum(K_wbs < T - 1), sum(K_wbs == T - 1), sum(K_wbs > T - 1)))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
---------- wbs -----------
avg loc error: 22.85, avg time: 0.6126012635231018
std loc error: 98.40034298720715, std time: 0.040011009746139685
K < K*: 1, K = K*: 99, K > K*: 0


In [15]:
loc_error_wbs

array([  5.,  10.,  40.,   5.,  10.,   5.,  20.,   5.,   5.,   5.,   5.,
        10.,   5.,  10.,   5., 995.,  10.,  10.,   5.,  20.,  20.,  20.,
        10.,  10.,  20.,  10.,  10.,  25.,   5.,  10.,  25.,  10.,  10.,
         5.,  50.,  10.,  10.,   5.,   5.,   5.,  50.,   5.,   5.,  50.,
        10.,  10.,   5.,  10.,   5.,  10.,   5.,   5.,  10.,  10.,   5.,
        10.,  25.,  55.,  10.,   5.,  10.,  10.,   5.,   5.,   5.,   5.,
         5.,  10.,  10.,  20.,  10.,  20.,   5.,  10.,   5.,   5.,  10.,
        10.,  20.,   5.,  55.,  25.,   5.,   5.,  10.,  10.,  20.,  40.,
        10.,  10.,  40.,  10.,  10.,  10.,  20.,  10.,   5.,  10.,  25.,
        10.])

In [16]:
import pickle
with open(path + 'wbs_mean_n' + str(n) + '_Delta' + str(Delta) + '_K' + str(T - 1) + '_grid' + str(grid_n) + '.pickle', 'wb') as f:
    pickle.dump([cp_best_list, param_best_list, loc_error_wbs, run_time_wbs, K_wbs], f)